### Создание признаков на основе тематического моделирования

В финальной ML модели не использовалось: признак не дал улучшения скоров.

In [8]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from functools import reduce

import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

### Загрузим сформированный ранее датасет, содержащий тексты сообщений

In [3]:
common = pd.read_csv('COMMON.csv')

In [4]:
common['value']

0                                                    xcxc
1                                                  dfxfgd
2                                                  dfhjdh
3                                          кажется ничему
4                                             кажется нет
                              ...                        
4692    Курсы по этой программе весьма интересны и име...
4693    Узнал много инструментов для разработки прилож...
4694    Да очень, много познавательного интересного уз...
4695    Есть полезная информация, которую стоит держат...
4696            структуру разработки мобильных приложений
Name: value, Length: 4697, dtype: object

## Topic Modeling: определение доминантной темы комментария 

In [5]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import nltk
from nltk.tokenize import ToktokTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import RussianStemmer
import re


# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

## Предобработка
Проведем очистку данных: единый регистр, удалим знаки препинания, цифры и т.п.
Также удалим все мусор и бессмыслицу. Оставим только корректно написанные русские слова. 
Произведем стемминг для сокращения количества уникальных словоформ.

Для проверки правописания используем библиотеку `pyenchant`.

Для установки на OS Windows: ```if you have installed PyEnchant from a wheel, you can download the hunspell dictionary files you need (both the .dic and .aff extensions) and put them inside /path/to/enchant/data/mingw<bits>/enchant/share/hunspell```

### Расширим стандартный словарь стоп-слов, удалив часто встречающиеся слова в данном конкретном датасете

#### Соберем все слова в одну переменную

In [92]:
text = ""
for i in common['value']:
    text += i+' '

#### Оставим только русские слова. Подсчитаем частоту каждого слова, исключая традиционные стоп слова (предлоги, союзы итп).

In [93]:
rus_stopwords = stopwords.words('russian')

text = re.sub('[^А-Яа-я]+', ' ', text)
all_words = nltk.tokenize.word_tokenize(text.lower())
all_words_no_stop = [w.lower() for w in all_words if w not in rus_stopwords]
freq_dict = nltk.FreqDist(w.lower() for w in all_words if w not in rus_stopwords)

#### Добавим топ 600 распространенных слов в список стоп-слов.

In [94]:
for i in range(600):
    common_word = freq_dict.most_common(600)[i][0]
    rus_stopwords.append(common_word)

In [95]:
import enchant
from enchant.checker import SpellChecker

d = enchant.DictWithPWL('ru')

def preprocess(line):
    char_regex = re.compile(r'[^а-яa-z]')
    line = char_regex.sub(' ', line.lower())
    
    chkr = SpellChecker(d)
    chkr.set_text(line)
    garbage = set([err.word for err in chkr])
    
    
    # filter out nonsense words and stem the rest
    tokenized = toktok.tokenize(line)
    filtered_str = ""
    for i in tokenized:
        if i not in garbage and i not in rus_stopwords:
            filtered = stemmer.stem(i)
            filtered_str += str(filtered + ' ')
            
    return filtered_str

In [96]:
# инициализация токенайзера и стеммера
toktok = ToktokTokenizer()
stemmer = RussianStemmer()

common['text'] = common['value'].apply(preprocess) 

In [97]:
texts = [i for i in common['text']]

In [98]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
data_words  = list(sent_to_words(texts))
print(data_words[9])

['нов', 'знан', 'тяжел', 'наш', 'студент']


In [99]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

# Form Bigrams
# data_words_bigrams = make_bigrams(data_words)

# Form Bigrams
data_words_trigrams = make_trigrams(data_words)

In [100]:
# LDA on trigrams

# Create Dictionary
id2word = corpora.Dictionary(data_words_trigrams)

# Create Corpus
texts = data_words_trigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[9])

[(4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]


In [101]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=300,
                                           passes=20,
                                           alpha='auto',
                                           per_word_topics=True)

### Посмотрим на ключевые слова по каждой из тем


In [102]:
lda_model.print_topics()

[(0,
  '0.016*"больш" + 0.012*"электрон" + 0.011*"сложн" + 0.010*"перв" + 0.010*"дистанцион" + 0.010*"системн" + 0.010*"изуч" + 0.010*"метод" + 0.009*"настроик" + 0.009*"автоматизирова"'),
 (1,
  '0.013*"общ" + 0.012*"курс" + 0.012*"структур" + 0.011*"код" + 0.011*"формул" + 0.010*"сред" + 0.009*"сетев" + 0.008*"программ" + 0.008*"маркетингов" + 0.007*"интересн"'),
 (2,
  '0.026*"использован" + 0.012*"техническ" + 0.012*"способ" + 0.011*"формул" + 0.010*"направлен" + 0.010*"современ" + 0.010*"статистик" + 0.010*"ос" + 0.009*"правильн" + 0.009*"результат"'),
 (3,
  '0.013*"расчет" + 0.012*"баз" + 0.012*"принцип" + 0.011*"цифров" + 0.011*"статистическ" + 0.010*"понят" + 0.010*"продвижен" + 0.010*"практическ" + 0.010*"функц" + 0.009*"процесс"'),
 (4,
  '0.015*"диаграмм" + 0.014*"таблиц" + 0.012*"применен" + 0.012*"язык" + 0.012*"нужн" + 0.011*"необходим" + 0.011*"график" + 0.009*"важн" + 0.009*"возможн" + 0.009*"алгоритм"')]

### Оценка качества модели

In [103]:
# Compute Perplexity (насколько хорошо модель предсказывает детали тестовой коллекции)
print('Perplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. the lower, the better.

# Compute Coherence Score (связность - от 0 до 1) # the higher, the better
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words_trigrams, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Perplexity:  -8.349178436266206
Coherence Score:  0.6353628809223728


### Визуализируем темы. В идеале они не должны пересекаться

Каждый пузырь представляет тему. Чем больше пузырь, тем больше распространена эта тема.

In [125]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

In [126]:
pyLDAvis.display(vis)

In [127]:
pyLDAvis.save_html(vis,'topics.html')

In [133]:
# если не отображается в ноутбуке, для интерактивной визуализации можно открыть topics.html в браузере.
import IPython
IPython.display.HTML(filename='topics.html')

#### Определим, какая тема главенствует в каждом из сообщений

In [105]:
dominant_topic = []
for i in range(len(corpus)):
    topics=[]
    for tupl in lda_model[corpus[i]][0]:
        topics.append(tupl[1])
    dominant_topic.append(np.argmax(topics))

In [106]:
pd.Series(dominant_topic).value_counts()

3    2185
4     750
1     674
0     573
2     515
dtype: int64

In [107]:
common['topic'] = dominant_topic

#### Агрегируем темы по юзеру: посмотрим на медианное значение

In [110]:
# агрегация тем по каждому юзеру
data_median_topic = common[['untiID','topic']].groupby(['untiID'],as_index=False).agg(np.median)
data_median_topic.columns = ['untiID','topic_max']


In [111]:
data_median_topic

,untiID,topic_max
0,4,3.0
1,6,3.0
2,85,3.0
3,99,3.0
4,103,3.0
...,...,...
1629,1057232,4.0
1630,1057244,3.0
1631,1057267,0.0
1632,1057370,3.0


In [ ]:
# можно смержить с данными диагностики и другими выделенными фичами и использовать как признак в модели.
# нам признак не помог улучшить скоры.
data_median_topic.to_csv('user_topics.csv',index=False)